In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('./data/6hour.csv')
df.columns = ['open_time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'close_time']
df.head()

,open_time,open_price,high_price,low_price,close_price,volume,close_time
0,1.502950e+09,302.40,312.00,301.90,308.95,2236.32270,1502971199999
1,1.502971e+09,308.95,312.18,298.21,308.33,2670.81200,1502992799999
2,1.502993e+09,308.33,311.53,298.96,302.00,1620.23441,1503014399999
3,1.503014e+09,302.00,306.47,295.19,304.70,2660.25471,1503035999999
4,1.503036e+09,304.70,311.79,303.21,307.78,1975.86747,1503057599999


In [8]:
# timestamp to datetime
df = df.drop(['open_time'], axis=1)
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
df['close_day'] = df['close_time'].dt.day
df['close_month'] = df['close_time'].dt.month
df['close_year'] = df['close_time'].dt.year
df = df[df['close_year'] < 2023]
df['target'] = np.where(df['close_price'].shift(-1) > df['close_price'], 1, 0)
df['open-close'] = df['open_price'] - df['close_price']
df['high-low'] = df['high_price'] - df['low_price']
df.tail()

,open_price,high_price,low_price,close_price,volume,close_time,close_day,close_month,close_year,target,open-close,high-low
7840,1195.55,1201.63,1194.02,1199.99,49869.5938,2022-12-30 23:59:59.999,30,12,2022,0,-4.44,7.61
7841,1199.98,1200.49,1191.66,1193.53,25985.0666,2022-12-31 05:59:59.999,31,12,2022,1,6.45,8.83
7842,1193.54,1200.23,1193.03,1197.20,27036.8761,2022-12-31 11:59:59.999,31,12,2022,1,-3.66,7.20
7843,1197.20,1208.46,1196.00,1200.78,53605.2053,2022-12-31 17:59:59.999,31,12,2022,0,-3.58,12.46
7844,1200.78,1205.61,1193.08,1196.13,32752.1331,2022-12-31 23:59:59.999,31,12,2022,0,4.65,12.53


In [22]:
def rule_based_decision(df, index):
    current_row = df.iloc[index]
    previous_row = df.iloc[index-1]
    previous_previous_row = df.iloc[index-2]

    if previous_row['open_price'] and previous_row['open_price'] > previous_previous_row['open_price']:
        return 1
    else: 
        return 0

In [23]:
prediction = []
for i in range(2, len(df)):
    prediction.append(rule_based_decision(df, i))
        
actual = df['target'].values[2:]
from sklearn.metrics import accuracy_score
accuracy_score(actual, prediction)


0.5131964809384164